In [2]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from xgboost import XGBClassifier

import time
# tmp sleep
# time.sleep(3600)
print('sleep done')
for f in sorted(glob.glob('../features/*.pkl')):
    print("'{}',".format(f))

sleep done
'../features/cnn_gru_muse_1_feat.pkl',
'../features/cnn_muse_1_feat.pkl',
'../features/cnn_muse_adj_1_feat.pkl',
'../features/cnn_muse_adj_2_feat.pkl',
'../features/gru_muse_1_feat.pkl',
'../features/lr_feat1.pkl',
'../features/lr_feat2.pkl',
'../features/lstm_1_feat.pkl',
'../features/lstm_2_feat.pkl',
'../features/lstm_muse_1_feat.pkl',
'../features/lstm_muse_adj_1_feat.pkl',
'../features/mnb_feat1.pkl',
'../features/mnb_feat2.pkl',
'../features/other_feat.pkl',
'../features/simple_rnn_muse_1_feat.pkl',
'../features/tfidf_feat1.pkl',
'../features/tfidf_feat2.pkl',


In [11]:
# load feats
chosen_fl = [
'../features/cnn_gru_muse_1_feat.pkl',
'../features/cnn_muse_1_feat.pkl',
'../features/cnn_muse_adj_1_feat.pkl',
'../features/cnn_muse_adj_2_feat.pkl',
'../features/gru_muse_1_feat.pkl',
#'../features/lr_feat1.pkl',
#'../features/lr_feat2.pkl',
'../features/lstm_1_feat.pkl',
'../features/lstm_2_feat.pkl',
'../features/lstm_muse_1_feat.pkl',
'../features/lstm_muse_adj_1_feat.pkl',
#'../features/mnb_feat1.pkl',
#'../features/mnb_feat2.pkl',
'../features/other_feat.pkl',
'../features/simple_rnn_muse_1_feat.pkl',
'../features/tfidf_feat1.pkl',
'../features/tfidf_feat2.pkl',
]
train_x,test_x = [],[]
for feat in chosen_fl:
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

file path ../features/cnn_gru_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_adj_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_adj_2_feat.pkl
(95851, 6) (226998, 6)
file path ../features/gru_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lr_feat1.pkl
(95851, 6) (226998, 6)
file path ../features/lr_feat2.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_muse_adj_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/mnb_feat1.pkl
(95851, 6) (226998, 6)
file path ../features/mnb_feat2.pkl
(95851, 6) (226998, 6)
file path ../features/other_feat.pkl
(95851, 16) (226998, 16)
file path ../features/simple_rnn_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/tfidf_feat1.pkl
(95851, 30) (226998, 30)
file path ../features/tfidf_feat2.pkl
(95851, 30) (226998, 30)
(95851, 142)
(95851, 142)


In [12]:
print(train_y[:10])
print(len(train_y))
print(sum(train_y[:,0]))

[[1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
95851
9237


In [13]:
from sklearn.model_selection import KFold
def simple_ens(model_name,k=3):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    test_pred = np.zeros((226998,6))
    all_train_loss_l,all_val_loss_l = 0,0
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        val_loss_l,train_loss_l = 0,0
        
        # train for each class
        for i in range(6):
            if model_name == 'lr':
                model = LogisticRegression()
            if model_name == 'xgb':
                model = XGBClassifier()
            model.fit(curr_x,curr_y[:,i])
            print(i)
            try:
                curr_train_loss = log_loss(curr_y[:,i],model.predict_proba(curr_x))
                curr_val_loss = log_loss(hold_out_y[:,i],model.predict_proba(hold_out_x))
                print(curr_train_loss,curr_val_loss)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
            except:
                pass
            curr_test_pred = model.predict_proba(test_x)[:,1]
            test_pred[:,i] += curr_test_pred
            
        # avg 6 class
        train_loss_l = train_loss_l/6
        val_loss_l = val_loss_l/6
        print('this fold avg train',train_loss_l,'avg val',val_loss_l)
        
        # avg k fold
        all_train_loss_l += train_loss_l/k
        all_val_loss_l += val_loss_l/k
        print('========================')
    test_pred = test_pred/k
    print('all train avg',all_train_loss_l,'all val avg',all_val_loss_l)
    return test_pred

print('done')
            
                
            

done


In [14]:
res = simple_ens('xgb',k=3)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = res
sample_submission.to_csv("../results/xgb_some_ens_csv.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

0
0.0826675323952 0.0864796840265
1
0.016995871375 0.0222824106571
2
0.0428286350435 0.0459591784184
3
0.0042312509528 0.00909187505543
4
0.0542161886509 0.0626274593261
5
0.0151218583383 0.0194458164913
this fold avg train 0.0360102227926 avg val 0.0409810706625
0
0.0812117921052 0.0906168496146
1
0.0172047117801 0.0218394561438
2
0.0415963089571 0.0483987466906
3
0.00456068529181 0.00865930221671
4
0.0553683152583 0.0601751280745
5
0.0150423866199 0.0204277274831
this fold avg train 0.0358307000021 avg val 0.0416862017039
0
0.0812902628141 0.0891499629047
1
0.0167684873669 0.0220828796832
2
0.0407209950515 0.0505747753542
3
0.00479845415064 0.00771469509962
4
0.0552044312248 0.0606818700086
5
0.0152534308462 0.0200478624305
this fold avg train 0.035672676909 avg val 0.0417086742468
all train avg 0.0358378665679 all val avg 0.0414586488711
         id     toxic  severe_toxic   obscene    threat    insult  \
0   6044863  0.001944      0.000065  0.000542  0.000072  0.000757   
1   61026